In [1]:
import pandas as pd
import numpy as np
import os 
import datetime
from dateutil.rrule import rrule, MONTHLY
import fastparquet

In [33]:
input_folder = 'D:\Tensor_Invest_Fund\data'

input_folder = '/media/john/Data/Tensor_Invest_Fund/data/'

data_files = os.listdir(os.path.join(os.getcwd(),input_folder))

stock = 'ES'

In [34]:
os.path.join(input_folder, f"Micro_Mini_Future_{stock}", f"MES_2020-2022.txt")

'/media/john/Data/Tensor_Invest_Fund/data/Micro_Mini_Future_ES/MES_2020-2022.txt'

In [35]:
#df_base = pd.read_parquet(os.path.join(input_folder, f"Micro_Mini_Future_{stock}", f"2019_Micro_Mini_Future_ES.parquet"))
df_delta = pd.read_csv(os.path.join(input_folder, f"Micro_Mini_Future_{stock}", f"M{stock}_2020_2022.txt"), sep = ',')

In [36]:
df_delta.tail()

,Date,Time,Open,High,Low,Close,Up,Down,Volume
703513,02/09/2022,22:56,4580.75,4581.25,4580.50,4580.75,36,35,71.0
703514,02/09/2022,22:57,4581.00,4581.25,4580.50,4581.25,14,24,38.0
703515,02/09/2022,22:58,4581.25,4581.75,4581.00,4581.25,25,17,42.0
703516,02/09/2022,22:59,4581.00,4581.50,4580.75,4581.00,13,38,51.0
703517,02/09/2022,23:00,4581.00,4582.00,4581.00,4581.50,65,36,101.0


In [37]:
def transform_raw_data(df, data_type = 'Tradestation'):

    if data_type == 'Ninjatrader':

        df['Date'] = df[0].apply(lambda x: str(x).split(' ')[0])
        df['Time'] = df[0].apply(lambda x: str(x).split(' ')[1])

        df.rename(columns = {1:'Open', 2: 'High', 3:'Low', 4:'Close', 5:'Vol'}, inplace = True)

        df['Date'] = pd.to_datetime( df['Date'], format = "%Y-%m-%d" )
        df['year'] = df['Date'].dt.year
        df['month'] = df['Date'].dt.month
        df['day'] = df['Date'].dt.day

        df['OI'] = np.nan

        df['hour'] = df['Time'].apply(lambda x: x[:2] )
        df['minute'] = df['Time'].apply(lambda x: x[2:4] )

        df.drop(columns = [0], inplace = True)

    elif data_type == 'Tradestation':

        columns = ['Date', 'Time', 'Open', 'High',
                     'Low', 'Close', 'Vol', 'OI', 'year',
                    'month', 'day', 'hour', 'minute']

        if ('Volume' in df.columns):
            df.rename(columns = {'Volume': 'Vol'}, inplace = True)

        df['Vol'] = df['Vol'].astype(float)
        df['Date'] = pd.to_datetime( df['Date'], format = "%m/%d/%Y" )
        df['year'] = df['Date'].dt.year
        df['month'] = df['Date'].dt.month
        df['day'] = df['Date'].dt.day

        if 'OI' in df.columns:
            df['OI'] = df['OI'].astype(float)
        else:
            df['OI'] = np.nan

        

        df['hour'] = df['Time'].apply(lambda x: x.split(":")[0] )
        df['minute'] = df['Time'].apply(lambda x: x.split(":")[1] )

        df = df.filter(columns)


    return df

In [38]:
df_delta = transform_raw_data(df_delta.copy(deep = True), data_type = 'Tradestation')

In [39]:
df_delta.tail()

,Date,Time,Open,High,Low,Close,Vol,OI,year,month,day,hour,minute
703513,2022-02-09,22:56,4580.75,4581.25,4580.50,4580.75,71.0,NaN,2022,2,9,22,56
703514,2022-02-09,22:57,4581.00,4581.25,4580.50,4581.25,38.0,NaN,2022,2,9,22,57
703515,2022-02-09,22:58,4581.25,4581.75,4581.00,4581.25,42.0,NaN,2022,2,9,22,58
703516,2022-02-09,22:59,4581.00,4581.50,4580.75,4581.00,51.0,NaN,2022,2,9,22,59
703517,2022-02-09,23:00,4581.00,4582.00,4581.00,4581.50,101.0,NaN,2022,2,9,23,00


# Delta Ingest

In [47]:
def append_delta(base_file, df_delta, base_year):
    """
    """

    df_base = pd.read_parquet(base_file)

    df_delta_clean = df_delta[(df_delta['Date'] > df_base['Date'].max()) & (df_delta['year'] == base_year )][df_base.columns].copy(deep = True)

    df_base = pd.concat([df_base, df_delta_clean], ignore_index = True )

    return df_base

In [48]:
base_year = 2021

base_file = os.path.join(input_folder,f'Micro_Mini_Future_{stock}' , f'2021_Micro_Mini_Future_{stock}.parquet')

df_base = append_delta(base_file, df_delta, base_year)

In [49]:
df_base.to_parquet(base_file, engine = 'fastparquet')

# New Unique Yearly File Creation

In [40]:
def create_new_year_file(df_delta, base_year, columns, new_file_loc):
    """
    New File Creation.
    """

    df_delta_clean = df_delta[(df_delta['year'] == base_year )][columns].copy(deep = True)

    df_delta_clean.to_parquet(new_file_loc, engine = 'fastparquet')

    return df_delta_clean

In [42]:
base_year = 2022

new_file_loc = os.path.join(input_folder, f'Micro_Mini_Future_{stock}' , f'{base_year}_Micro_Mini_Future_{stock}.parquet')

columns = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Vol', 'OI', 'year',
       'month', 'day', 'hour', 'minute']

create_new_year_file(df_delta, base_year, columns, new_file_loc)

,Date,Time,Open,High,Low,Close,Vol,OI,year,month,day,hour,minute
665122,2022-01-03,00:01,4770.75,4778.00,4770.25,4772.75,1507.0,NaN,2022,1,3,00,01
665123,2022-01-03,00:02,4772.75,4775.50,4771.50,4774.50,846.0,NaN,2022,1,3,00,02
665124,2022-01-03,00:03,4774.25,4776.00,4772.75,4773.00,512.0,NaN,2022,1,3,00,03
665125,2022-01-03,00:04,4773.25,4774.00,4771.75,4772.00,436.0,NaN,2022,1,3,00,04
665126,2022-01-03,00:05,4772.00,4773.25,4771.25,4773.25,348.0,NaN,2022,1,3,00,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...
703513,2022-02-09,22:56,4580.75,4581.25,4580.50,4580.75,71.0,NaN,2022,2,9,22,56
703514,2022-02-09,22:57,4581.00,4581.25,4580.50,4581.25,38.0,NaN,2022,2,9,22,57
703515,2022-02-09,22:58,4581.25,4581.75,4581.00,4581.25,42.0,NaN,2022,2,9,22,58
703516,2022-02-09,22:59,4581.00,4581.50,4580.75,4581.00,51.0,NaN,2022,2,9,22,59


In [52]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278423 entries, 0 to 278422
Data columns (total 13 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   Date    278423 non-null  datetime64[ns]
 1   Time    278423 non-null  object        
 2   Open    278423 non-null  float64       
 3   High    278423 non-null  float64       
 4   Low     278423 non-null  float64       
 5   Close   278423 non-null  float64       
 6   Vol     278423 non-null  float64       
 7   OI      278423 non-null  float64       
 8   year    278423 non-null  int64         
 9   month   278423 non-null  int64         
 10  day     278423 non-null  int64         
 11  hour    278423 non-null  object        
 12  minute  278423 non-null  object        
dtypes: datetime64[ns](1), float64(6), int64(3), object(3)
memory usage: 27.6+ MB


# Generate Yearly Files 

In [55]:
def generate_yearly_parquet(df, stock_name, input_folder, start_year = None, end_year = None):

    start_year = start_year if start_year is not None else df['Date'].min().year
    end_year = end_year if end_year is not None else df['Date'].max().year

    stock_folder  = os.path.join(input_folder, stock_name)

    if not os.path.exists(stock_folder):
        os.makedirs(stock_folder)

    for year in range(start_year, end_year+1):

        yearly_data = df[df['year'] == year].copy(deep = True)

        file_name = f'{year}_{stock_name}.parquet'

        file_storage = os.path.join(stock_folder , file_name)

        yearly_data.to_parquet(file_storage, engine = 'fastparquet')

        print(file_storage)


In [56]:
# Valid Names:
# Micro_Mini_Future_ES
# Micro_Mini_Future_NQ
stock_name = 'Micro_Mini_Future_NQ'

generate_yearly_parquet(df_new, stock_name, input_folder, start_year=1999, end_year=2021)

../../data\Micro_Mini_Future_NQ\1999_Micro_Mini_Future_NQ.parquet
../../data\Micro_Mini_Future_NQ\2000_Micro_Mini_Future_NQ.parquet
../../data\Micro_Mini_Future_NQ\2001_Micro_Mini_Future_NQ.parquet
../../data\Micro_Mini_Future_NQ\2002_Micro_Mini_Future_NQ.parquet
../../data\Micro_Mini_Future_NQ\2003_Micro_Mini_Future_NQ.parquet
../../data\Micro_Mini_Future_NQ\2004_Micro_Mini_Future_NQ.parquet
../../data\Micro_Mini_Future_NQ\2005_Micro_Mini_Future_NQ.parquet
../../data\Micro_Mini_Future_NQ\2006_Micro_Mini_Future_NQ.parquet
../../data\Micro_Mini_Future_NQ\2007_Micro_Mini_Future_NQ.parquet
../../data\Micro_Mini_Future_NQ\2008_Micro_Mini_Future_NQ.parquet
../../data\Micro_Mini_Future_NQ\2009_Micro_Mini_Future_NQ.parquet
../../data\Micro_Mini_Future_NQ\2010_Micro_Mini_Future_NQ.parquet
../../data\Micro_Mini_Future_NQ\2011_Micro_Mini_Future_NQ.parquet
../../data\Micro_Mini_Future_NQ\2012_Micro_Mini_Future_NQ.parquet
../../data\Micro_Mini_Future_NQ\2013_Micro_Mini_Future_NQ.parquet
../../data

In [57]:
df_new[df_new['year'] == 2021].tail(10)

,Date,Time,Open,High,Low,Close,Vol,OI,year,month,day,hour,minute
6097664,2021-10-14,22:45,15064.50,15064.75,15062.25,15063.75,54.0,76.0,2021,10,14,22,45
6097665,2021-10-14,22:46,15064.00,15064.75,15062.50,15064.25,88.0,62.0,2021,10,14,22,46
6097666,2021-10-14,22:47,15064.00,15065.25,15063.50,15064.50,37.0,48.0,2021,10,14,22,47
6097667,2021-10-14,22:48,15064.25,15065.50,15064.00,15064.75,25.0,29.0,2021,10,14,22,48
6097668,2021-10-14,22:49,15064.50,15065.00,15063.50,15063.50,24.0,40.0,2021,10,14,22,49
6097669,2021-10-14,22:50,15063.50,15063.50,15061.00,15061.00,31.0,102.0,2021,10,14,22,50
6097670,2021-10-14,22:51,15061.00,15066.25,15060.75,15065.25,163.0,53.0,2021,10,14,22,51
6097671,2021-10-14,22:52,15065.50,15067.00,15065.50,15067.00,24.0,22.0,2021,10,14,22,52
6097672,2021-10-14,22:53,15066.50,15067.00,15064.25,15065.00,20.0,40.0,2021,10,14,22,53
6097673,2021-10-14,22:54,15065.00,15065.00,15064.25,15064.50,2.0,6.0,2021,10,14,22,54
